# **CSI 382 - Data Mining and Knowledge Discovery**

# **Lab 7 - Neural Networks**

The inspiration for neural networks was the recognition that complex learning systems
in animal brains consisted of closely interconnected sets of neurons. Although a particular neuron may be relatively simple in structure, dense networks of interconnected neurons could perform complex learning tasks such as classiﬁcation and pattern recognition. The human brain, for example, contains approximately $10^{11}$ neurons, each connected on average to $10,000$ other neurons, making a total of $1,000,000,000,000,000=10^{15}$ synaptic connections.

**Definition**

Artiﬁcial neural networks (hereafter, neural networks) represent an attempt at a very basic level to imitate the type of nonlinear learning that occurs in the networks of neurons found in nature.

# **Dataset for Lab 7**

**Data Set Information:**

The examined group comprised kernels belonging to three different varieties of wheat: Kama, Rosa and Canadian, 70 elements each, randomly selected for
the experiment. High quality visualization of the internal kernel structure was detected using a soft X-ray technique. It is non-destructive and considerably cheaper than other more sophisticated imaging techniques like scanning microscopy or laser technology. The images were recorded on 13x18 cm X-ray KODAK plates. Studies were conducted using combine harvested wheat grain originating from experimental fields, explored at the Institute of Agrophysics of the Polish Academy of Sciences in Lublin.

The data set can be used for the tasks of classification and cluster analysis.

**Attribute Information:**

To construct the data, seven geometric parameters of wheat kernels were measured:
1. area A,
2. perimeter P,
3. compactness C = 4*pi*A/P^2,
4. length of kernel,
5. width of kernel,
6. asymmetry coefficient
7. length of kernel groove.
All of these parameters were real-valued continuous.

**Relevant Papers:**

M. Charytanowicz, J. Niewczas, P. Kulczycki, P.A. Kowalski, S. Lukasik, S. Zak, 'A Complete Gradient Clustering Algorithm for Features Analysis of X-ray Images', in: Information Technologies in Biomedicine, Ewa Pietka, Jacek Kawa (eds.), Springer-Verlag, Berlin-Heidelberg, 2010, pp. 15-24.

The dataset can be found here in this [URL](https://drive.google.com/file/d/1ErhaHBOWTTf7648GnAXDI4EhqnP-cimi/view?usp=sharing)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **Loading the dataset**

In [ ]:
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp

In [ ]:
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

In [ ]:
# Test Backprop on Seeds dataset
seed(1)
#load and prepare data
filename = '/content/drive/MyDrive/wheat-seeds.csv'
dataset = load_csv(filename)

In [ ]:
dataset[:10]

## **Dataset Preprocessing**

As a neural network does not understand anything rather than numbers, then let's convert every entity into numbers first.

In [ ]:
# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())#dataset e unwanted value thakle strip kore strip() diye

# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	print(unique)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup


In [ ]:
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)

# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)# last column jodi(dataset,7) ota specific detaset er jono

# **Input-Output Encoding**

One possible drawback of neural networks is that all attribute values must be encoded in a standardized manner, taking values between zero and 1, even for categorical variables. Later, when we examine the details of the back-propagation algorithm, we shall understand why this is necessary.

For now, however, how does one go about standardizing all the attribute values?

## **Continuous Variables**

For continuous variables, this is not a problem, as we discussed in Lecture 2.
We may simply apply the \textit{min–max normalization}:
\begin{equation*}
    X* = \frac{X-min(X)}{range(X)} = \frac{X-min(X)}{max(X) - min(X)}
\end{equation*}
This works well as long as the minimum and maximum values are known and all
potential new data are bounded between them. Neural networks are somewhat robust to minor violations of these boundaries. If more serious violations are expected,
certain ad hoc solutions may be adopted, such as rejecting values that are outside the
boundaries, or assigning such values to either the minimum or maximum value.

## **Categorical Variables**

Categorical variables are more problematical, as might be expected. If the number of possible categories is not too large, one may use indicator (ﬂag) variables.


For example, many data sets contain a $gender$ attribute, containing values $female$, $male$,
and $unknown$. Since the neural network could not handle these attribute values in their
present form, we could, instead, create $indicator$ variables for female and male. Each
record would contain values for each of these two $indicator$ variables. Records for
females would have a value of $1$ for $female$ and 0 for $male$, while records for males
would have a value of $0$ for $female$ and $1$ for $male$. Records for persons of unknown
gender would have values of 0 for $female$ and 0 for $male$.


In general, categorical
variables with $k$ classes may be translated into $k - 1$ indicator variables, as long as
the deﬁnition of the indicators is clearly deﬁned.

In [ ]:
# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	stats = [[min(column), max(column)] for column in zip(*dataset)]
	return stats

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)-1):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [ ]:
# normalize input variables
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

In [ ]:
dataset[:10]

## **Output**

With respect to output, we shall see that neural network output nodes always
return a continuous value between zero and 1 as output.

**How can we use such continuous output for classiﬁcation?**

Many classiﬁcation problems have a dichotomous result, an up-or-down decision, with only two possible outcomes. For example, “Is this customer about to leave our company’s service?” For dichotomous classiﬁcation problems, one option is to use a single output node , with a threshold value set a priori which would separate the classes, such as “leave” or “stay.” For example, with the threshold of “leave if output $\geq$ 0.67,” an output of 0.72 from the output node would classify that record as likely to leave the company’s service.

# **Neural Network**

## **Number of Nodes**

The number of input nodes usually depends on the number and type of attributes
in the data set.

The number of hidden layers, and the number of nodes in each hidden
layer, are both conﬁgurable by the user.

One may have more than one node in the
output layer, depending on the particular classiﬁcation task at hand.

### **Hidden Layer**

Since more nodes in the
hidden layer increases the power and ﬂexibility of the network for identifying complex
patterns, one might be tempted to have a large number of nodes in the hidden layer.
On the other hand, an overly large hidden layer leads to over-fitting, memorizing
the training set at the expense of generalizability to the validation set. If over-fitting
is occurring, one may consider reducing the number of nodes in the hidden layer;
conversely, if the training accuracy is unacceptably low, one may consider increasing
the number of nodes in the hidden layer.

### **Input Layer**

The input layer accepts inputs from the data set, such as attribute values, and
simply passes these values along to the hidden layer without further processing. Thus,
the nodes in the input layer do not share the detailed node structure that the hidden
layer nodes and the output layer nodes share.

# **Neural Networks - Working Procedure**



First, a combination function (usually summation, $\sum$) produces a linear combination of the node inputs and the connection weights into a single scalar value, which we will term $net$. Thus, for a given node $j$,

$ net_j = \sum_{i}{W_{ij}x_{ij} = W_{0j}x_{0j} + W_{1j}x_{1j}+ \dots +W_{Ij}x_{Ij}} $

where $x_{ij}$ represents the $i^{th}$ input to node $j$, $W_{ij}$ represents the weight associated with the $i^{th}$ input to node $j$, and there are $I + 1$ inputs to node $j$. Note that $x_1 , x_2 , \dots , x_I$ represent inputs from upstream nodes, while $x_0$ represents a constant input, analogous to the constant factor in regression models, which by convention uniquely takes the value $x_{0j} = 1$. Thus, each hidden layer or output layer node $j$ contains an “extra” input equal to a particular weight $W_{0j}x_{0j} = W_{0j} $, such as $W_{0B}$ for node $B$.

In [ ]:
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

In [ ]:
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] -= l_rate * neuron['delta']

For example, for node $A$ in the hidden layer, we have
\begin{equation*}
    \begin{split}
        net_A &= \sum_{i}{W_{iA}x_{iA} = W_{0A}(1) + W_{1A}x_{1A}+ W_{2A}x_{2A}+ W_{3A}x_{3A}}\\
        &= 0.5 + 0.6(0.4) + 0.8(0.2) + 0.6(0.7) = 1.32
    \end{split}
\end{equation*}
Within node $A$, this combination function $net_A = 1.32$ is then used as an input to an activation function.

In biological neurons, signals are sent between neurons when the
combination of inputs to a particular neuron cross a certain threshold, and the neuron
“ﬁres.”

This is nonlinear behavior, since the ﬁring response is not necessarily linearly
related to the increment in input stimulation. Artiﬁcial neural networks model this
behavior through a **nonlinear activation function**.

# **Sigmoid Activation Function**

The most common activation function is the sigmoid function:
\begin{equation*}
    \begin{split}
        y &= \frac{1}{1+e^{-x}}
    \end{split}
\end{equation*}
where $e$ is base of natural logarithms, equal to about 2.718281828. Thus, within
node $A$, the activation would take $net_A = 1.32$ as input to the sigmoid activation
function, and produce an output value of $y = \frac{1}{(1 + e^{-1.32} )} = 0.7892$.

Node $A$’s
work is done (for the moment), and this output value would then be passed along the
connection to the output node $Z$, where it would form (via another linear combination)
a component of $net_Z$.

In [ ]:
# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation

# Transfer neuron activation
def transfer(activation):
	return 1.0 / (1.0 + exp(-activation))

But before we can compute $net_Z$, we need to ﬁnd the contribution of node $B$.
From the values in Figure, we have,

\begin{equation*}
    \begin{split}
        net_B &= \sum_{i}{W_{iB}x_{iB} = W_{0B}(1) + W_{1B}x_{1B}+ W_{2B}x_{2B}+ W_{3B}x_{3B}}\\
        &= 0.7 + 0.9(0.4) + 0.8(0.2) + 0.4(0.7) = 1.5
    \end{split}
\end{equation*}
Then,
\begin{equation*}
    \begin{split}
        f(net_B) &= \frac{1}{1+e^{-1.5}} = 0.8176
    \end{split}
\end{equation*}

Node $Z$ then combines these outputs from nodes $A$ and $B$, through $net_Z$, a weighted
sum, using the weights associated with the connections between these nodes.

Note
that the inputs $x_i$ to node $Z$ are not data attribute values but the outputs from the sigmoid functions from upstream nodes:
\begin{equation*}
    \begin{split}
        net_Z &= \sum_{i}{W_{iZ}x_{iZ} = W_{0Z}(1) + W_{AZ}x_{AZ}+ W_{BZ}x_{BZ}}\\
        &= 0.5 + 0.9(0.7892) + 0.9(0.8176) = 1.9461
    \end{split}
\end{equation*}
Finally, $net_Z$ is input into the sigmoid activation function in node $Z$, resulting in
\begin{equation*}
    \begin{split}
        f(net_Z) &= \frac{1}{1+e^{-1.9461}} = 0.8750
    \end{split}
\end{equation*}

This value of 0.8750 is the output from the neural network for this ﬁrst pass through
the network, and represents the value predicted for the target variable for the ﬁrst
observation.

**Why use the sigmoid function?**

Because it combines nearly linear behavior, curvilinear
behavior, and nearly constant behavior, depending on the value of the input.

Figure \ref{fig:sigmoid1} shows the graph of the sigmoid function $ y = f (x) = \frac{1}{(1 + e^{-x} )}$, for $-5 < x < 5$
[although $f(x)$ may theoretically take any real-valued input].

Through much of the
center of the domain of the input $x$ (e.g., $-1 < x < 1$), the behavior of $f(x)$ is nearly
linear. As the input moves away from the center, $f(x)$ becomes curvilinear. By the
time the input reaches extreme values, $f(x)$ becomes nearly constant.


# **Back propagation**

**How does a neural network learn?**


Neural networks represent a supervised learning
method, requiring a large training set of complete records, including the target
variable. As each observation from the training set is processed through the network,
an output value is produced from the output node.

This output value is then compared to the actual value
of the target variable for this training set observation, and the error (actual - output)
is calculated. This prediction error is analogous to the residuals in regression models.

To measure how well the output predictions fit the actual target values, most neural
network models use the sum of squared errors:

$ {SSE} = \sum_{{records}}{\sum_{{output nodes}}{{(actual - output)}^2}} $

where the squared prediction errors are summed over all the output nodes and over
all the records in the training set.

The problem is therefore to construct a set of model weights that will minimize
the SSE. In this way, the weights are analogous to the parameters of a regression
model. The “true” values for the weights that will minimize SSE are unknown, and
our task is to estimate them, given the data. However, due to the nonlinear nature of
the sigmoid functions permeating the network, there exists no closed-form solution
for minimizing SSE as exists for least-squares regression.

In [ ]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

# **Gradient Descent Method**

We must therefore turn to optimization methods, specifically gradient-descent methods,
to help us find the set of weights that will minimize SSE.


Suppose that we have a
set (vector) of $m$ weights $w = w_0,w_1,w_2, \dots , w_m$ in our neural network model and
we wish to find the values for each of these weights that, together, minimize SSE.
We can use the gradient descent method, which gives us the direction that we should
adjust the weights in order to decrease SSE. The gradient of SSE with respect to the
vector of weights $w$ is the vector derivative:

$
\nabla{SSE(W)} =  \left[  \frac{ \partial{SSE}} {\partial w_0}, \frac{ \partial{SSE}} {\partial w_1},\dots, \frac{ \partial{SSE}} {\partial w_m}  \right]
$

that is, the vector of partial derivatives of SSE with respect to each of the weights.

In [ ]:
# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

In [ ]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

# **Back Propagation Rules**

The back-propagation algorithm takes the prediction error (actual - output) for a
particular record and percolates the error back through the network, assigning partitioned responsibility for the error to the various connections. The weights on these
connections are then adjusted to decrease the error, using gradient descent.

Using the sigmoid activation function and gradient descent, Mitchell derives
the back-propagation rules as follows:

$ w_{ij,new} = w_{ij,current} + \Delta w_{ij} \quad \text{ where,}
\quad \Delta w_{ij} = \eta \delta_{j} x_{ij} $

Now we know that $\eta$ represents the learning rate and $x_{ij}$ signiﬁes the $i^{{th}}$ input to node
$j$, but what does $\delta_j$ represent? The component $\delta_j$ represents the responsibility for a
particular error belonging to node $j$. The error responsibility is computed using the
partial derivative of the sigmoid function with respect to $net_j$ and takes the following
forms, depending on whether the node in question lies in the output layer or the hidden
layer:

\begin{equation*}
    \delta_j =
\begin{cases}
    {output}_j (1 - {output}_j )({actual}_j - {output}_j ) & \text{for output layer nodes}\\
    {output}_j (1 -{output}_j) \sum\limits_{{downstream}}{W_{jk}\delta_j} & \text{for hidden layer nodes}          
\end{cases}
\end{equation*}

where $\sum_{{downstream}}{W_{jk}\delta_j}$ refers to the weighted sum of the error responsibilities for
the nodes downstream from the particular hidden layer node. (For the full derivation,
see Mitchell \cite{mitchell1997machine}.)

In [ ]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(neuron['output'] - expected[j])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])


In [ ]:
# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_hidden, n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)

In [ ]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)

# **Termination Criteria**

The neural network algorithm would then proceed to work through the training data
set, record by record, adjusting the weights constantly to reduce the prediction error.

It may take many passes through the data set before the algorithm’s termination
criterion is met. What, then, serves as the termination criterion, or stopping criterion?
If training time is an issue, one may simply set the number of passes through the
data, or the amount of real-time the algorithm may consume, as termination criteria.

However, what one gains in short training time is probably bought with degradation
in model efficacy.

In [ ]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

In [ ]:
# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

# **Learning Rate**

Recall that the learning rate $\eta$, $0 < \eta < 1$, is a constant chosen to help us move the
network weights toward a global minimum for SSE.

**However, what value should $\eta$ take? How large should the weight adjustments be?**

When the learning rate is very small, the weight adjustments tend to be very
small. Thus, if $\eta$ is small when the algorithm is initialized, the network will probably
take an unacceptably long time to converge. Is the solution therefore to use large
values for $\eta$? Not necessarily. Suppose that the algorithm is close to the optimal
solution and we have a large value for $\eta$. This large $\eta$ will tend to make the algorithm
overshoot the optimal solution.

Consider Figure 7.5, where $W^*$ is the optimum value for weight $W$, which
has current value $W_{current}$. According to the gradient descent rule, $\Delta w_{current} = -\eta(\partial SSE/\partial w_{current})$, $W_{current}$ will be adjusted in the direction of $W^*$. But if the learning
rate $\eta$, which acts as a multiplier in the formula for  $\Delta w_{current}$, is too large, the new
weight value $W_{new}$ will jump right past the optimal value $W^*$, and may in fact end up
farther away from $W^*$ than $W_{current}$.

In [ ]:
n_folds = 5
l_rate = 0.1
n_epoch = 600
n_hidden = 21
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

# **That's all for today!**

# **Tasks**

Go to this [url](https://drive.google.com/file/d/1PU1AQVCgnFL2XbgIJRl3TwkHY5Ag3qIv/view?usp=sharing) and download the data first. In order to know more about the dataset please refer to these links - [UCI/iris](https://archive.ics.uci.edu/ml/datasets/Iris), or [Kaggle/iris_dataset](https://www.kaggle.com/uciml/iris).

**The "species" field refers to the Predicted attribute: class of iris plant.**

 Now try to do the following:

1. Apply an ANN to find/predict/classify the class of IRIS plants.
2. Apply different termination criteria to see if the accuracy changes.